In [ ]:
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
from torch import nn,optim
path = r"C:\Users\kanna\Desktop\GUVI\4.Data\Breast Cancer\train"
transformation = transforms.Compose(
    [
        transforms.Resize((640,640)),
        transforms.ToTensor()
    ]
)# Line5 to 9 (lets resize the image and convert it to tensor(numerical value of pixel representation))
data = datasets.ImageFolder(path,transform=transformation) # Loading the data and converting to tensor
# data.classes
num_classes = len(data.classes) # finding the number of classes, not using now, but needed in case of multi class classifiation.

loader = DataLoader(data, batch_size=64,shuffle=True) #convert  the entire dataset in batches of size 64 each, in 1 batch we have 64 images and 64 labels.

class bcnn(nn.Module):
    def __init__(self,input_channels):
        super(bcnn,self).__init__()
        self.cv1 = nn.Conv2d(input_channels,out_channels=32,kernel_size=3)# 32 = channels, 3 = Kernel, Default stride = 1
                                                 # 640+0-3+1 = 638*638
                                                 # kernel will be constructed by the model, no need to mention.
        self.p1 = nn.MaxPool2d(kernel_size=3,stride=3) # 3 = Kernel size, stride = 3
                                    # 638-3/3+1 --> 212*212*32 pixels
                                    # reduce the number of pixels and maintain only the important features
                                    # used for computation efficiency
        self.flatten = nn.Flatten() # transforms into single dimension, 2 dimension(tabular data) into single dimension
        self.fc1 = nn.Linear(212*212*32,128)
        self.fc2 = nn.Linear(128,64)
        self.fc3 = nn.Linear(64,1) # 1 at the last layer because of binary classification
        self.sigmoid = nn.Sigmoid()

    def forward(self,x):
        x = self.cv1(x)
        x = self.p1(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.sigmoid (self.fc3(x))
        return x 
img, label = data[0]
input_channels = img.shape[0]
model = bcnn(input_channels) # object of the neural network
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())
num_epoch = 10
for epoch in range(1,num_epoch+1):
    total_loss = 0
    model.train()
    for x,y in loader:
        y = y.float().unsqueeze(1) # to create as list of list
        optimizer.zero_grad() # resetting the gradiants
        output = model(x) # forward function is applied
        loss = criterion(output,y) # loss in output layer
        loss.backward() # compute error for each neuron in previous layers
        optimizer.step() # update weights
        total_loss = total_loss+loss.item()
    print (f"{epoch}/{num_epoch}: Loss --> {total_loss}")
    
